# Pipeline 2 - Stockfish engine evaluation

## Inputs
- The extracted files from previous pipeline 1

## Output
- A csv file

## CSV file columns
For each row of the csv file, the output contains:
- **Basic game info**
    - WhiteUsername
    - BlackUsername
    - WhiteElo
    - BlackElo
    - EloDifference
    - TimeControl
    - Opening
    - GameId
- **Each step info**
    - MoveId
    - RemainingTime
    - MovePlayed
    - MovePlayedEval
    - ProcessTime
    - NumberofNodes
- **Algorithm prediction** ($n$ from $1$ to $5$)
    - best_move_$n$,
    - best_score_$n$,
    - ProcessTime_$n$,
    - NumberofNodes_$n$,

WhiteUsername,BlackUsername,WhiteElo,BlackElo,EloDifference,TimeControl,Opening,GameId,
MoveId,RemainingTime,MovePlayed,MovePlayedEval,ProcessTime,NumberofNodes,

best_move_1,best_score_1,ProcessTime_1,NumberofNodes_1,
best_move_2,best_score_2,ProcessTime_2,NumberofNodes_2,
best_move_3,best_score_3,ProcessTime_3,NumberofNodes_3,
best_move_4,best_score_4,ProcessTime_4,NumberofNodes_4,
best_move_5,best_score_5,ProcessTime_5,NumberofNodes_5

---

## Import APIs

In [2]:
# import julia libraries
using PyCall
using CSV
using DataFrames

# import python chess library
@pyimport chess
cp = pyimport("chess.pgn")
ce = pyimport("chess.engine")

PyObject <module 'chess.engine' from '/home/ubuntu/.local/lib/python3.10/site-packages/chess/engine.py'>

## Engine path

In [3]:
# Constants
const STOCKFISH_PATH = "/usr/local/bin/stockfish"

const INPUT_DATA_PATH = "/data/pipeline1_exported/" # file type: .pgn and .csv file
const OUTPUT_DATA_PATH = "/data/pipeline2_exported/" # file type: .csv file

"/usr/local/bin/stockfish"

## Methods

In [6]:
function initialize_stockfish()
    stockfish = ce.SimpleEngine.popen_uci(STOCKFISH_PATH)
    return stockfish
end

# Placeholder analyze_game function
function analyze_game(game, stockfish)
    # Placeholder data structure, replace with actual game analysis logic
    return DataFrame(WhiteUsername = "placeholder", BlackUsername = "placeholder", WhiteElo = 0,
                     BlackElo = 0, EloDifference = 0, TimeControl = "placeholder", Opening = "placeholder",
                     GameId = "placeholder", MoveId = 0, RemainingTime = 0, MovePlayed = "placeholder",
                     MovePlayedEval = 0, ProcessTime = 0, NumberofNodes = 0, best_move_1 = "placeholder",
                     best_score_1 = 0, ProcessTime_1 = 0, NumberofNodes_1 = 0, best_move_2 = "placeholder",
                     best_score_2 = 0, ProcessTime_2 = 0, NumberofNodes_2 = 0, best_move_3 = "placeholder",
                     best_score_3 = 0, ProcessTime_3 = 0, NumberofNodes_3 = 0, best_move_4 = "placeholder",
                     best_score_4 = 0, ProcessTime_4 = 0, NumberofNodes_4 = 0, best_move_5 = "placeholder",
                     best_score_5 = 0, ProcessTime_5 = 0, NumberofNodes_5 = 0)
end

function process_games(file_path, stockfish)
    pgn_file = open(file_path)
    game_data = DataFrame()

    while true
        game = cp.read_game(pgn_file)
        if game === nothing
            break
        end

        game_analysis_data = analyze_game(game, stockfish)
        append!(game_data, game_analysis_data)
    end

    close(pgn_file)
    return game_data
end


process_games (generic function with 1 method)

## Execution

In [7]:
main()

"test_output.csv"